# Projet Apprentissage Artificiel
---
Yoann Menanteau, Thomas Saliba, Jérémy Bezamat

---

## Contexte :
Ce projet a été réalisé dans le cadre d'un cours sur l'apprentissage artificiel de la promotion robotique de Bordeaux INP. Il consiste a essayer de trouver la fréquence d'un signal périodique à l'aide d'un réseau de neurones de type ESN (echo state network) fournie par le dépôt [reservoirpy](https://github.com/neuronalX/reservoirpy).

## Démarche :
Notre démarche consiste dans un premier temps à essayer d'entrainer un réseau de neurones permettant de trouver la fréquence d'un signal périodique basique composé d'une seule sinusoide. Cette première étape nous as permis de bien prendre en main _reservoirpy_. Par la suite, nous avons entrainé un deuxième réseau de neurones mais qui permet cette fois ci de trouver la fréquence d'un signal périodique plus complexe composé d'une somme de sinusoide.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ESN
from random import randint

In [2]:
def set_seed(seed=None):
    """Making the seed (for random values) variable if None"""

    # Set the seed
    if seed is None:
        import time
        seed = int((time.time()*10**6) % 4294967295)
    try:
        np.random.seed(seed)
    except Exception, e:
        print "!!! WARNING !!!: Seed was not set correctly."
        print "!!! Seed that we tried to use: "+str(seed)
        print "!!! Error message: "+str(e)
        seed = None
    print "Seed used for random values:", seed
    return seed


## Set a particular seed for the random generator (for example seed = 42), or use a "random" one (seed = None)
# NB: reservoir performances should be averaged accross at least 30 random instances (with the same set of parameters)
seed = 42 #None #42
set_seed(seed) #random.seed(seed)

time_steps = 600 # Nombre d'échantillons par période
nb_periods = 5 # Nombre de périodes
n_input_by_freq = 10 # Nombre d'exemples pour chaque fréquence
random_coeff = 1000. 
min_offsetx = -1/random_coeff
min_offsety = -1/random_coeff
max_offsetx = 1/random_coeff
max_offsety = 1/random_coeff
min_amplitude = -1/random_coeff
max_amplitude = 1/random_coeff
# chances = 5# une chance sur 2 qu'une harmonique ne soit pas changée


# 30 seems to be enough for initLen with leak_rate=0.3 and reservoir size (resSize) = 300
initLen = 30 # number of time steps during which internal activations are washed-out during training
# we consider trainLen including the warming-up period (i.e. internal activations that are washed-out when training)
max_freq = initLen + 500 # number of time steps during which we train the network
#testLen = max_freq # number of time steps during which we test/run the network


Seed used for random values: 42


La première étape est de créer un script permettant de générer de manière aléatoire des signaux périodiques sinusoidaux de manière à entrainer le réseau. On génére une seed pour l'aléatoire en se basant sur le temps de sort à avoir une génération la moins répétable possible.

Le nombre d'échantillons par période est fixé par la variable `time_steps`. Le nombre de periode est fixé par la variable `nb_periods`. Les variables qui suivent (min_offsetx, min_offsety, etc...) permettent de rajouter un peu de bruits aux tests afin d'être proche d'une situation réelle.

In [3]:
def create_sine(frequency, offsetx, offsety, size, A):
    return A * np.sin(2 * np.pi * frequency * np.linspace(0, 1./frequency*size, time_steps*size)+offsetx)+offsety


train_out = list(range(1, max_freq))*n_input_by_freq
train_in = np.array([create_sine(freq, np.random.uniform(min_offsetx,max_offsetx), np.random.uniform(min_offsety,max_offsety), nb_periods, np.random.uniform(min_amplitude,max_amplitude)) for freq in train_out])

train_out = np.asarray(train_out)
temp = []
for freq in train_out:
    temp.append([freq])
train_out = np.asarray(temp)

print "train_in, train_out dimensions", train_in.shape, train_out.shape

test_out = list(range(1, max_freq))*n_input_by_freq
test_in = np.array([create_sine(freq, np.random.uniform(min_offsetx,max_offsetx), np.random.uniform(min_offsety,max_offsety), nb_periods, np.random.uniform(min_amplitude,max_amplitude)) for freq in train_out])
plt.plot(test_in[150])
test_out = np.asarray(test_out)
temp = []
for freq in test_out:
    temp.append([freq])
test_out = np.asarray(temp)
print "test_in, test_out dimensions", test_in.shape, test_out.shape



train_in, train_out dimensions (5290, 3000) (5290, 1)
test_in, test_out dimensions (5290, 3000) (5290, 1)


AttributeError: 'module' object has no attribute 'to_rgba'

## Première partie: Sinusoides simple

Dans cette première partie, nous nous sommes concentrés sur la génération d'un réseau pouvant extraire la fréquence d'une simple sinusoide. Pour créer les sinusoides, nous avons implémenté la fonction `create_sine`. Ensuite, le script créé un ensemble d'entrainement grâce aux tableaux `train_in` et `train_out` (le premier servant à stocker l'ensemble des valeurs du sinus discrétisé et le second stockant la fréquence de chacun de ces sinus). 

On génere `n_input_by_freq` par fréquence en prenant toutes les fréquences allant de 0 à `max_freq`. Chacun de ces sinus est généré avec des valeurs aléatoires pour l'amplitude, la phase et un décalage en y. 

Les parties du code comme celle ci-dessous permettent de mettre en forme les données avant de les envoyer au réseau de neurones:
    
    train_out = np.asarray(train_out)
    temp = []
    for freq in train_out:
        temp.append([freq])
    train_out = np.asarray(temp)
    

Ce script affiche également un exemple d'une de ces sinusoides générées. 



In [4]:
n_inputs = 1
input_bias = True # add a constant input to 1
n_outputs = 1
n_reservoir = 300 # number of recurrent units
leak_rate = 0.3 # leaking rate (=1/time_constant_of_neurons)
spectral_radius = 1.25 # Scaling of recurrent matrix
input_scaling = 1. # Scaling of input matrix
proba_non_zero_connec_W = 0.2 # Sparsity of recurrent matrix: Perceptage of non-zero connections in W matrix
proba_non_zero_connec_Win = 1. # Sparsity of input matrix
proba_non_zero_connec_Wfb = 1. # Sparsity of feedback matrix
regularization_coef =  1e-8 #None # regularization coefficient, if None, pseudo-inverse is use instead of ridge regression
# out_func_activation = lambda x: x

N = n_reservoir#100
dim_inp = n_inputs #26

### Generating random weight matrices with custom method
W = np.random.rand(N,N) - 0.5
if input_bias:
    Win = np.random.rand(N,dim_inp+nb_periods*time_steps) - 0.5
else:
    Win = np.random.rand(N,dim_inp+nb_periods*time_steps-1) - 0.5
Wfb = np.random.rand(N,n_outputs) - 0.5

## delete the fraction of connections given the sparsity (i.e. proba of non-zero connections):
mask = np.random.rand(N,N) # create a mask Uniform[0;1]
W[mask > proba_non_zero_connec_W] = 0 # set to zero some connections given by the mask
mask = np.random.rand(N,Win.shape[1])
Win[mask > proba_non_zero_connec_Win] = 0

## SCALING of matrices
# scaling of input matrix
Win = Win * input_scaling
# scaling of recurrent matrix
# compute the spectral radius of these weights:
print 'Computing spectral radius...',
original_spectral_radius = np.max(np.abs(np.linalg.eigvals(W)))
print "default spectral radius before scaling:", original_spectral_radius
# rescale them to reach the requested spectral radius:
W = W * (spectral_radius / original_spectral_radius)
print "spectral radius after scaling", np.max(np.abs(np.linalg.eigvals(W)))

reservoir = ESN.ESN(lr=leak_rate, W=W, Win=Win, input_bias=input_bias, ridge=regularization_coef, Wfb=None, fbfunc=None)

Computing spectral radius... default spectral radius before scaling: 2.35001160885
spectral radius after scaling 1.25


### Création du réseau de neurones

Le réseau a besoin d'une certaine configuration. Il a besoin de poids synaptiques. On distingue les poids des synapses en entrées et les autres. Les synapses connectent les neurones. Le fait de connecter ou non est aléatoire est définie selon `proba_non_zero_connec_W`, `proba_non_zero_connec_Win`, `proba_non_zero_connec_Wfb`. Les valeurs des synapses `Win` (entrées) et `W` sont définies aléatoirement. `n_inpu` défini le nombre d'entrées et `n_output` le nombre de sorties. Cette dernière est fixée à un puisqu'on cherche à obtenir une fréquence par signal. Le nombre d'input dépend du nombre d'échantillons du signal lui même dépendant de la variable `time_steps`. 

L'état du réseau dépend de cette formule:
x(t+ 1) = (1−α)x(t) +αf(Winu(t+ 1) + Wx(t))
y(t) = Wout*x(t)
avec x(t) l'état du réservoir selon le temps t, α un hyper-paramètre leak rate, u(t) l'entrée en fonction du temps et y(t) la sortie en fonction du temps.

In [5]:
internal_trained = reservoir.train(inputs=[train_in,], teachers=[train_out,], wash_nr_time_step=initLen, verbose=False)

On donne ensuite au réseau les données d'entrainement

In [6]:
output_pred, internal_pred = reservoir.run(inputs=[test_in,], reset_state=False)
errorLen = len(test_out[:]) #testLen #2000
mse = np.mean((test_out[:] - output_pred[0])**2) # Mean Squared Error: see https://en.wikipedia.org/wiki/Mean_squared_error
rmse = np.sqrt(mse) # Root Mean Squared Error: see https://en.wikipedia.org/wiki/Root-mean-square_deviation for more info
nmrse_mean = abs(rmse / np.mean(test_out[:])) # Normalised RMSE (based on mean)
nmrse_maxmin = rmse / abs(np.max(test_out[:]) - np.min(test_out[:])) # Normalised RMSE (based on max - min)
print("\n********************")
print("Errors computed over %d time steps" % (errorLen))
print("\nMean Squared error (MSE):\t\t%.4e" % (mse) )
print("Root Mean Squared error (RMSE):\t\t%.4e\n" % rmse )
print("Normalized RMSE (based on mean):\t%.4e" % (nmrse_mean) )
print("Normalized RMSE (based on max - min):\t%.4e" % (nmrse_maxmin) )
print("********************\n")


********************
Errors computed over 5290 time steps

Mean Squared error (MSE):		3.2253e+04
Root Mean Squared error (RMSE):		1.7959e+02

Normalized RMSE (based on mean):	6.7771e-01
Normalized RMSE (based on max - min):	3.4014e-01
********************



### Résultats

Les résultats de la première étape (avec des sinusoides simples) après avoir données au réseau les données de test sont présentés ci-dessus. Nous n'avons malheureusement pas pu tester notre réseau avec un grand nombre de données d'entrainement car nous n'avions pas de machine assez performante.

In [7]:
def create_sine_with_noise(nbperiods):
    signal = 0
    freqs = []
    nb_sin = 2
    for i in range(0, nb_sin):
        freq = randint(1, max_freq)
        freqs.append(freq)
    freq_min = min(freqs)
    freq_max = max(freqs)
    for i in range(0, nb_sin):
        s = np.random.uniform(min_amplitude,max_amplitude) * np.sin(2 * np.pi * freqs[i] * \
            np.linspace(0, (1./freq_min)*nb_periods, time_steps*nb_periods)+np.random.uniform(min_offsetx,max_offsetx))\
                +np.random.uniform(min_offsety,max_offsety)
        signal += s
    return signal, freq_min

## Deuxième partie: Somme de sinusoides

Dans cette partie, on génére cette fois ci des signaux périodiques plus complexes pour l'ensemble des données d'entrainement et de test. Ces signaux sont des sommes de sinusoides (avec `nb_sin` sinusoides par signal) créées par la fonction `create_sine_with_noise` présentée ci-dessus. 

In [8]:
def list_to_a(l, d):
    l = np.asarray(l)
    temp = []
    for x in l:
        if d == 1:
            temp.append([x])
        else:
            temp.append(x)
    return np.asarray(temp)

n_input = 10*max_freq # pour avoir même nombre de données d'entrainement que le test précédent
train_in = []
train_out = []
for i in range(1, max_freq):
    sig, freq = create_sine_with_noise(nb_periods)
    train_in.append(sig)
    train_out.append(freq)
    
train_in = list_to_a(train_in, 2)
train_out = list_to_a(train_out, 1)

print "train_in, train_out dimensions", train_in.shape, train_out.shape

test_in = []
test_out = []
for i in range(1, max_freq):
    sig, freq = create_sine_with_noise(nb_periods)
    test_in.append(sig)
    test_out.append(freq)

test_in = list_to_a(test_in, 2)
test_out = list_to_a(test_out, 1)
plt.plot(test_in[10])

print "test_in, test_out dimensions", test_in.shape, test_out.shape



train_in, train_out dimensions (529, 3000) (529, 1)
test_in, test_out dimensions (529, 3000) (529, 1)


AttributeError: 'module' object has no attribute 'to_rgba'

In [9]:
internal_trained = reservoir.train(inputs=[train_in,], teachers=[train_out,], wash_nr_time_step=initLen, verbose=False)

In [10]:
output_pred, internal_pred = reservoir.run(inputs=[test_in,], reset_state=False)
errorLen = len(test_out[:]) #testLen #2000
mse = np.mean((test_out[:] - output_pred[0])**2) # Mean Squared Error: see https://en.wikipedia.org/wiki/Mean_squared_error
rmse = np.sqrt(mse) # Root Mean Squared Error: see https://en.wikipedia.org/wiki/Root-mean-square_deviation for more info
nmrse_mean = abs(rmse / np.mean(test_out[:])) # Normalised RMSE (based on mean)
nmrse_maxmin = rmse / abs(np.max(test_out[:]) - np.min(test_out[:])) # Normalised RMSE (based on max - min)
print("\n********************")
print("Errors computed over %d time steps" % (errorLen))
print("\nMean Squared error (MSE):\t\t%.4e" % (mse) )
print("Root Mean Squared error (RMSE):\t\t%.4e\n" % rmse )
print("Normalized RMSE (based on mean):\t%.4e" % (nmrse_mean) )
print("Normalized RMSE (based on max - min):\t%.4e" % (nmrse_maxmin) )
print("********************\n")


********************
Errors computed over 529 time steps

Mean Squared error (MSE):		2.6425e+06
Root Mean Squared error (RMSE):		1.6256e+03

Normalized RMSE (based on mean):	9.4143e+00
Normalized RMSE (based on max - min):	3.2708e+00
********************



### Résultats

On retrouve ici aussi les résultats du réseau de neurones après lui avoir envoyé les données de test mais cette fois ci avec des signaux périodiques plus complexes.

# Conclusion

Ce projet a permis de se familiariser avec les réseaux de neurones de type ESN. Cela a permis de se confronter aux problématiques courantes lors de l'utilisation d'un réseau de neurones dans un but précis. Choisir le format d'entrée, considérer les métriques intéressantes, quel taille de corpus choisir, etc.. Toutes ces questions se posent lorsque l'on cherche à utiliser un réseau de neurones.
Lors de ce projet, un générateur de sinusoïdes a été créé de manière à pouvoir fabriquer un signal périodique simple (une sinusoide) ou plus complexe (somme de sinusoides). Avec ce code, il est possible de générer n'importe quel signal périodique étant donné que tout signal peut se résumer à une somme de sinusoides (série de Fourier). Une fois le corpus créé, le réseau de neurones doit être entrainé puis testé. La compléxité réside souvent dans la bonne qualité du corpus d'entrainement. La phase d'entrainement quand à elle ne nécessite qu'une phase de paramètrage.
Il est intéressant de se poser la question de l'utilité d'un réseau de neurones dans un contexte comme celui de ce projet. Le problème de passage d'onde vers fréquence est résolu mathématiquement, c'est la transformée de Fourier. Ce calcul est très présent en traitement du signal et donc il éxiste bon nombre de puces et d'algorithmes optimisés pour résoudre le problème de manière rapide et robuste. L'utilisation d'un réseau de neurones pour résoudre un problème qui possède déjà une résolution mathématique connue n'est pas forcément intéressante en terme pratique. Le FPGA est une soultion alternative permettant de potentiellement battre les meilleurs scores de rapidité du réseau de neurones car la stucture est totalement adaptative aux besoins de l'utilisateur. L'implémentation de réseaux de neurones sur FPGA est d'ailleurs au coeur de la recherche actuelle.